INSTALAÇÃO DE DEPENDÊNCIAS E BIBLIOTECAS QUE SERÃO NECESSÁRIAS

In [4]:
!pip install pydantic
!pip install pydantic[email]

IMPORTAÇÕES DAS BIBLIOTECAS QUE SERÃO UTILIZADAS NO CÓDIGO

In [11]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
    ValidationError,
)

CRIAÇÃO DE EXPRESSÕES REGULARES PARA VALIDAR NOMES E SENHAS

In [12]:
# Define uma regra para as senhas, a senha deve possuir no mínimo 8 caracteres,
# sendo pelo menos 1 letra maiúscula, 1 letra minúscula e 1 número.
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[A-Z])(?=.*[a-z])(?=.*\d).{8,}$")
# Define uma regra para os nomes, o nome deve possuir no mínimo 2 caracteres.
VALID_NAME_REGEX = re.compile(r"^[A-Za-z]{2,}$")

CRIAÇÃO DE UMA CLASSE DE CLASSIFICAÇÃO DE PERMISSÕES

In [13]:
# Essa classe serve para designar uma função para o Usuário
# O ordem de "importância" segue a ordem de declaração
# Ou seja, User < Author < Editor < Admin < SuperAdmin
# O que significa que um SuperAdmin é também um Admin, Editor, Author e User
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

CRIAÇÃO DA CLASSE USUÁRIO COM VALIDAÇÕES E SERIALIZAÇÃO PERSONALIZADAS PARA OS DADOS

In [ ]:
class User(BaseModel):
    # O uso do 'str' é para garantir que o campo seja uma string, 
    # além disso ele oferce um exemplo de um valor válido para essa variável
    name: str = Field(examples=["Example"])
    # O uso do 'EmailStr' é para garantir que o campo seja um email
    # O 'Field' é utilizado para adicionar metadados e resrições ao campo, 
    # servindo como uma indicação do formato esperado para esse campo
    email: EmailStr = Field(
        # Ele oferce um exemplo de um valor válido para essa variável
        examples=["user@arjancodes.com"],
        # Aqui é feita uma descrição do que é esperado para esse campo
        description="The email address of the user",
        #O parâmetro Frozen é utilizado para garantir que o objeto não seja alterado após a sua criação
        frozen=True,
    )
    #O uso do 'SecretStr' é para garantir que o campo seja uma senha, 
    #ou seja, caso ela seja exibida em algum lugar, ela será exibida como '********'
    # A opção 'exclude' é utilizada para garantir que, ao serializar esse objeto, 
    # a senha não será incluída
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    #O atributo role é utilizado para designar a função do usuário
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )
    
    #Esses 'fields_validator' são validações personalizadas feitas para garantir que os dados inseridos sejam válidos
    @field_validator("name")
    @classmethod
    #Como é um método de classe, ele recebe a classe como primeiro argumento e o valor a ser validado como segundo argumento
    #O retorno desse método é uma string
    def validate_name(cls, v: str) -> str:
        # Aqui é feita uma verificação, a partir de correspondência com a "regra" da REGEX imposta, 
        # para garantir que o nome inserido seja válido
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letter and be at least 2 characters long"
            )
        return v


    #Aqui é feita uma validação para garantir que o valor inserido para o usuário seja válido
    #Como o mode dessa validação é 'before', ela é feita antes da validação padrão do Pydantic
    @field_validator("role", mode="before")
    @classmethod
    #Como é um método de classe, ele recebe a classe como primeiro argumento 
    #e o segundo parâmetro é o valor a ser validado, que pode ser um inteiro, uma string ou um Role
    #O retorno desse método é um Role, ou seja, um valor de função 
    def validate_role(cls, v: int | str | Role) -> Role:
        #Esse dicionário é utilizado para mapear os valores possíveis para a função
        #Se v for um inteiro, ele é convertido para um Role, se for uma string, 
        #ele é convertido para um Role - por meio do nome da role -, se for um Role, ele é retornado
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            #Aqui é feita a conversão do valor inserido para um Role
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {",".join([x.name for x in Role])}"
                )
        
    #Aqui é feita uma validação para garantir que o valor inserido para o usuário seja válido
    #Como o mode dessa validação é 'before', ela é feita antes da validação padrão do Pydantic
    @model_validator(mode="before")
    @classmethod
     #Como é um método de classe, ele recebe a classe como primeiro argumento 
     #e o valor a ser validado como segundo argumento, em forma de dicionário  
     #contendo  os dados do usuário- onde 
     #as chaves são strings e os valores podem ser de qualquer tipo.
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        #Aqui é feita uma verificação para garantir que o nome e a senha foram inseridos
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        #Aqui é feita uma verificação para garantir que a senha não contenha o nome do usuário
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        #Aqui é feita uma verificação para garantir que a senha inserida seja
        #válida, ou seja, contenha 8 caracteres, 1 maiúscula, 1 minúscula e 1 número
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        #Aqui é feita uma criptografia da senha inserida, para melhorar a segurança
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v
    
    #Aqui é feita uma validação para garantir que o valor inserido para o usuário seja válido
    #Como o mode dessa validação é 'after', ela é feita após a validação padrão do Pydantic
    @model_validator(mode="after")
    #Como é um método de instância, ele recebe a instância da classe como primeiro argumento
    #e o valor a ser validado como segundo argumento
    def validate_user_post(self, v: Any) -> Self:
        # Aqui é feita uma verificação para garantir que apenas o Arjan pode ser um admin
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    # Esse método é utilizado para serializar a função do usuário
    @field_serializer("role", when_used="json")
    @classmethod
    # Como é um método de classe, ele recebe a classe como primeiro argumento 
    # e o valor a ser serializado como segundo argumento
    def serialize_role(cls, v) -> str:
        return v.name

    # Esse método é utilizado para serializar o usuário, atua durante a serialização
    # do objeto completo para JSON
    @model_serializer(mode="wrap", when_used="json")
    # Como é um método de instância, ele recebe a instância da classe como primeiro argumento
    # e o segundo argumento é um objeto de informações, que contém informações sobre a serialização
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        # Aqui é feita uma verificação para garantir que nada 
        # que tenha restrições seja incluído na serialização
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

CRIAÇÃO DA FUNÇÃO MAIN (PRINCIPAL)

In [ ]:
def main() -> None:
    # Dicionário de dados para um usuário
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    # Aqui é feita a validação e criação do Usuário user
    user = User.model_validate(data)
    if user:
        # Aqui é feita a serialização dos dados para um dicionário
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        # Aqui é feita a serialização dos dados para um JSON
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        # Aqui é feita a serialização dos dados para um JSON, excluindo a role
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        # Aqui é feita a serialização dos dados para um dicionário, incluindo a role
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")

EXECUÇÃO DA FUNÇÃO MAIN 

In [ ]:
if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}
